# Q2.1 - Systematic Encoding Matrix  $G$ 

In [7]:
import numpy as np

def rref(M):
    """
    Computes the Row Reduced Echelon Form (RREF) of matrix M using XOR.
    Returns the RREF matrix and the list of pivot column indices.
    """
    M = M.copy()
    rows, cols = M.shape
    pivot_cols = []
    current_row = 0
    
    for j in range(cols):
        if current_row >= rows:
            break
        
        pivot_row = -1
        for i in range(current_row, rows):
            if M[i, j] == 1:
                pivot_row = i
                break
        
        if pivot_row != -1:
            if pivot_row != current_row:
                M[[current_row, pivot_row]] = M[[pivot_row, current_row]]
            # Record the pivot column
            pivot_cols.append(j)
            
            # Use XOR to zero out all other 1s in column
            for i in range(rows):
                if i != current_row and M[i, j] == 1:
                    # M[i] = M[i] XOR M[current_row]
                    M[i] = M[i] ^ M[current_row]
            
            current_row += 1
            
    return M, pivot_cols

def build_systematic_encoder(H_original):
    """
    Builds the systematic LDPC encoder G using the procedure H_hat = [P | I] and G = [I_K / P].
    """
    H = H_original.copy()
    M, N = H.shape
    K = N - M  # K = message length
    
    # Compute RREF of H to find pivots
    H_rref, pivot_cols = rref(H)

    all_cols = list(range(N))
    non_pivot_cols = [c for c in all_cols if c not in pivot_cols]
    
    # Determine Permutation Order: [Non-Pivots] (P-part) + [Pivots] (I-part)
    perm_order = non_pivot_cols + pivot_cols
    
    # Apply Permutation to the RREF matrix to get the final H_hat = [P | I]
    H_hat = H_rref[:, perm_order]

    P = H_hat[:, :K]  # P is the first K columns of H_hat
    I_K = np.eye(K, dtype=int)
    G = np.vstack([I_K, P])
    
    return H_hat, G


# Test Input
H_matrix = np.array([
    [1, 1, 1, 1, 0, 0],
    [0, 0, 1, 1, 0, 1],
    [1, 0, 0, 1, 1, 0]
], dtype=int)

print("Original H:")
print(H_matrix)

# Run the function
H_hat, G = build_systematic_encoder(H_matrix)

print("-" * 40)
print("System Parameters: N=6, M=3, K=3")
print("-" * 40)

print("\nPermuted and Row-Reduced H (H_hat = [P | I]):")
print(H_hat)

print("\nSystematic Generator Matrix G (G = [I_K / P]):")
print(G)

# Verification: H_hat * G should be 0
check = (H_hat @ G) % 2
print("\nVerification (H_hat @ G % 2):")
print(check)
print(f"All zero? {np.all(check == 0)}")

Original H:
[[1 1 1 1 0 0]
 [0 0 1 1 0 1]
 [1 0 0 1 1 0]]
----------------------------------------
System Parameters: N=6, M=3, K=3
----------------------------------------

Permuted and Row-Reduced H (H_hat = [P | I]):
[[1 1 0 1 0 0]
 [1 1 1 0 1 0]
 [1 0 1 0 0 1]]

Systematic Generator Matrix G (G = [I_K / P]):
[[1 0 0]
 [0 1 0]
 [0 0 1]
 [1 1 0]
 [1 1 1]
 [1 0 1]]

Verification (H_hat @ G % 2):
[[0 0 0]
 [0 0 0]
 [0 0 0]]
All zero? True


---
We must ensure $H$ is in RREF form ($H_{RREF} = [ P \ | \ I ]$) to solve the system of linear equations. This explicitly separates the message bits from the parity bits, allowing us to directly construct $G$ without doing any further algebra. Example $H$ in the question is not in RREF. So, we have implemented a function to convert $H$ to $H_{RREF}$

We verify this orthogonality (that $H_{RREF} \times G = 0$) below:

$H_{RREF} = [ P \ | \ I ]$

$H_{RREF} \times G = [ P \ | \ I ] \times \begin{bmatrix} I \\ P \end{bmatrix}$

   $= (P \times I) + (I \times P)$

   $= P + P = 0$ (since we are operating in binary where $x+x=0$)

we must have our $\hat{H}$ be equal to $H_{RREF}$



# Q2.2 - Factor Graph for Matrix $H$

<img src="images/GM2.2.jpg" width="300">

Here is the factor graph from the H given in Question 2.1

A factor graph for an LDPC code is a bipartite graph

Each check node $c_i$ connects to variable node $x_j$ if the corresponding entry in the matrix row is 1 ($H_{ij} = 1$).

$
H =
\begin{bmatrix}
1 & 1 & 1 & 1 & 0 & 0 \\
0 & 0 & 1 & 1 & 0 & 1 \\
1 & 0 & 0 & 1 & 1 & 0
\end{bmatrix}
$
